In [1]:
import numpy as np
import vectorbt as vbt
from utils.data_processing import load_data
from random_forest.random_forest import infer

PEROID_TYPE = '15minutely' # daily, minutely, 15minutely

if PEROID_TYPE == 'daily':
    start_date_test = "2023-01-01"
    end_date_test = "2024-01-01"
elif PEROID_TYPE == '15minutely':
    start_date_test = "2023-05-01"
    end_date_test = "2024-01-01"
elif PEROID_TYPE == 'minutely':
    start_date_test = "2023-08-06"
    end_date_test = "2024-01-01"

tics = [
    "AAPL",
    "GOOG",
    "AMZN",
    "NVDA",
    "TSLA",
    "AMD",
    "INTC",
    "ATVI",
    "PYPL",
    "TTD",
    "EA",
    ]

target_items = [
    'FR1TF',
    # 'FR3TF',
    # 'FR5TF',
    # 'FR10TF'
]

input_items = [
    'RSI5',
    'RSI10',
    'RSI20',
    'RSI60',
    
    'PD_SMA5_Close',
    'PD_SMA10_Close',
    'PD_SMA20_Close',
    'PD_SMA60_Close',
    
    'PD_SMA5_SMA10', 
    'PD_SMA5_SMA20', 
    'PD_SMA5_SMA60',
    
    'PD_SMA10_SMA20',
    'PD_SMA10_SMA60',
    
    'PD_SMA20_SMA60',  
    
    'PD_RSI5_RSI10',
    'PD_RSI5_RSI20',
    'PD_RSI5_RSI60',
    
    'PD_RSI10_RSI20',
    'PD_RSI10_RSI60',
    
    'PD_RSI20_RSI60',
    
    'SL3CLOSE',
    'SL5CLOSE',
    'SL10CLOSE',
    'SL20CLOSE',
    'SL60CLOSE',
    
    'SL3SMA5',
    'SL5SMA5',
    'SL10SMA5',
    'SL20SMA5',
    'SL60SMA5',
    
    'SL3SMA10',
    'SL5SMA10',
    'SL10SMA10',
    'SL20SMA10',
    'SL60SMA10',
    
    'SL3SMA20',
    'SL5SMA20',
    'SL10SMA20',
    'SL20SMA20',
    'SL60SMA20',
    
    'SL3SMA60',
    'SL5SMA60',
    'SL10SMA60',
    'SL20SMA60',
    'SL60SMA60',
    
    'SL3RSI5',
    'SL5RSI5',
    'SL10RSI5',
    'SL20RSI5',
    'SL60RSI5',
    
    'SL3RSI10',
    'SL5RSI10',
    'SL10RSI10',
    'SL20RSI10',
    'SL60RSI10',
    
    'SL3RSI20',
    'SL5RSI20',
    'SL10RSI20',
    'SL20RSI20',
    'SL60RSI20',
]

pf = {}
price_data = {}
entries = {}
exits = {}
x_train, y_train, x_test, y_test = {}, {}, {}, {}

for tic in tics:
    print('-'*50 + tic +'-'*50)
    path = "data/" + PEROID_TYPE + "/" + tic + ".csv"
    
    data = load_data(path, start_date_test, end_date_test, indicators=True)
    data = data.dropna()
    
    price_data[tic] = data.copy().squeeze()
    price_data[tic] = price_data[tic]['Close']
    print('len(price_data[tic])', len(price_data[tic]))
    
    model_data = data[input_items].copy()
    
    signal = np.ones((len(price_data[tic])), dtype=bool)
    for target_item in target_items:
        new_signal = infer(model_data, 'random_forest/model/' + tic + '/' + PEROID_TYPE + '/' + target_item + '.joblib')
        signal = np.logical_and(signal, new_signal)
    
    signal_next = np.roll(signal,1)
    
    entries[tic] = np.logical_and(signal,np.logical_not(signal_next))
    exits[tic] = np.logical_and(np.logical_not(signal),signal_next)
    
    pf[tic] = vbt.Portfolio.from_signals(
        price_data[tic], 
        entries[tic], 
        exits[tic], 
        # freq="D", 
        # fees=0.01, 
        init_cash=1000)



--------------------------------------------------AAPL--------------------------------------------------


c:\Users\edmundlylee\Documents\GitHub\ib-trading\utils\data_processing.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['PD_'+ i + '_' + j] = get_percentage_difference(df[i], df[j])
c:\Users\edmundlylee\Documents\GitHub\ib-trading\utils\data_processing.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['PD_'+ i + '_' + j] = get_percentage_difference(df[i], df[j])
c:\Users\edmundlylee\Documents\GitHub\ib-trading\utils\data_processing.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

len(price_data[tic]) 1772


In [2]:
for tic in tics:
    print(tic)
    pf[tic].plot().show()
    print(pf[tic].stats())

AAPL


c:\Users\edmundlylee\.conda\envs\ib\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\edmundlylee\.conda\envs\ib\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\edmundlylee\.conda\envs\ib\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\edmundlylee\.conda\envs\ib\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2023-05-01 21:30:00
End                           2023-08-09 01:15:00
Period                                       1772
Start Value                                1000.0
End Value                             1051.189975
Total Return [%]                         5.118997
Benchmark Return [%]                      5.23591
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         5.420496
Max Drawdown Duration                       429.0
Total Trades                                  293
Total Closed Trades                           293
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            50.511945
Best Trade [%]                           4.380265
Worst Trade [%]                         -2.462276
Avg Winning Trade [%]                    0.273203
Avg Losing Trade [%]                    -0.252667
